<a href="https://colab.research.google.com/github/travelingshoez/amazon_vine_analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [908 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://arc

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-10 16:14:03--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.07MB/s    in 0.2s    

2023-03-10 16:14:04 (6.07 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

### Create DataFrames to match tables

In [33]:
# Read in the Review dataset as a DataFrame
from pyspark import SparkFiles
from pyspark.sql.functions import to_date
review_df = df.select('review_id','review_headline', 'review_body', 'review_date')
review_df.show()

+--------------+--------------------+--------------------+-----------+
|     review_id|     review_headline|         review_body|review_date|
+--------------+--------------------+--------------------+-----------+
|R1KKOXHNI8MSXU|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|R26SP2OPDK4HT7|Favorite for wint...|I love this dress...| 2014-03-04|
| RWQEDYAX373I1|Great Socks for t...|Nice socks, great...| 2015-07-12|
|R231YI7R4GPF6J|          Slick hat!|I bought this for...| 2015-06-03|
|R3KO3W45DD0L1K|I would do it again!|Perfect dress and...| 2015-06-12|
|R1C4QH63NFL5NJ|          Five Stars|Excellent for my ...| 2014-04-22|
|R2GP65O1U9N7BP|            Love it!|Raw is the only w...| 2015-07-28|
|R3O29CT5MQQ3XQ|         Three Stars|        A bit large.| 2015-07-10|
|R1ZECD2AA8QFF6|          Five Stars|          Great fit!| 2015-08-09|
|R2S79GCF6J89OA|    Not my favorite.|Shirt a bit too l...| 2014-05-24|
|R1CBCUGNP37MPK|The Jockey Women'...|The Jockey Women'...| 2015-06-10|
|R3NU4

In [6]:
# Create the customers_table DataFrame

from pyspark.sql.functions import col, count
customers_df = df.groupBy('customer_id').agg(count('customer_id').alias('customer_count'))
customers_df.show()




+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13780696|             1|
|   48489393|             8|
|   43380682|             1|
|   17133428|             1|
|    9731896|             2|
|    1797768|             1|
|   27446622|             1|
|     120706|             3|
|   15915947|             2|
|   37520920|             1|
|   43920023|             9|
|   14309190|             8|
|   22316505|             1|
|   43949054|             2|
|   21102747|             1|
|   19021463|             1|
|   16075459|             8|
|   51475782|             2|
|   11555870|             5|
|   35471383|             3|
+-----------+--------------+
only showing top 20 rows



In [19]:
# Create the products_table DataFrame and drop duplicates. 
from pyspark.sql.functions import count
products_df = df.groupBy('product_title', 'product_category').agg(count('*').alias('count')).dropDuplicates()
products_df.show()



+--------------------+----------------+-----+
|       product_title|product_category|count|
+--------------------+----------------+-----+
|Victorious Mens T...|         Apparel|   81|
|I'LL BE BACK JESU...|         Apparel|    1|
|Dreamall Women's ...|         Apparel|    1|
|Misses Fold Over ...|         Apparel|    6|
|Elesa Miracle Hai...|         Apparel|    2|
|WEEL ARROW Womens...|         Apparel|    1|
|BB Dakota Women's...|         Apparel|    1|
|Women's Long Slee...|         Apparel|    1|
|OMG Woman's Broca...|         Apparel|    1|
|WHITE APPAREL Wom...|         Apparel|    4|
|VINA VINO Women's...|         Apparel|    1|
|Fashion Secrets S...|         Apparel|    7|
|Twenty One Pilots...|         Apparel|    1|
|Women's Elegant C...|         Apparel|    5|
|Mexico - Mexican ...|         Apparel|    1|
|Expression Tees T...|         Apparel|    1|
|Favebridal Women'...|         Apparel|    1|
|Caramel Cantina J...|         Apparel|    1|
|Oberon Design Bol...|         App

In [39]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
from pyspark.sql.functions import to_date
review_id_df = df.select('review_id', 'review_date')
review_id_df = df.select('review_id', to_date('review_date', 'yyyy-MM-dd').alias('review_date'))
review_id_df.show()





+--------------+-----------+
|     review_id|review_date|
+--------------+-----------+
|R1KKOXHNI8MSXU| 2013-01-14|
|R26SP2OPDK4HT7| 2014-03-04|
| RWQEDYAX373I1| 2015-07-12|
|R231YI7R4GPF6J| 2015-06-03|
|R3KO3W45DD0L1K| 2015-06-12|
|R1C4QH63NFL5NJ| 2014-04-22|
|R2GP65O1U9N7BP| 2015-07-28|
|R3O29CT5MQQ3XQ| 2015-07-10|
|R1ZECD2AA8QFF6| 2015-08-09|
|R2S79GCF6J89OA| 2014-05-24|
|R1CBCUGNP37MPK| 2015-06-10|
|R3NU4WUR5ZNV1V| 2013-09-06|
|R32EPCJ3XF8LGE| 2014-11-17|
|R1XIBC6WQBW31M| 2013-05-12|
|R1P11MVDZ65LMH| 2014-02-14|
|R1OJA3DJL0VDDK| 2014-10-20|
|R1THWA5YRJLOOF| 2015-07-05|
| R738LCNRSJVXP| 2015-07-29|
|R1N3Z13931J3O9| 2015-01-01|
|R1LBNTP7E8N89Z| 2015-02-12|
+--------------+-----------+
only showing top 20 rows



In [41]:
# Create the vine_table. DataFrame
vine_df = df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
vine_df.show()



+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|
|R26SP2OPDK4HT7|          5|            1|          2|   N|
| RWQEDYAX373I1|          5|            0|          0|   N|
|R231YI7R4GPF6J|          5|            0|          0|   N|
|R3KO3W45DD0L1K|          5|            0|          0|   N|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|
|R2GP65O1U9N7BP|          5|            0|          0|   N|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|
|R2S79GCF6J89OA|          3|            0|          0|   N|
|R1CBCUGNP37MPK|          5|            0|          0|   N|
|R3NU4WUR5ZNV1V|          5|            0|          0|   N|
|R32EPCJ3XF8LGE|          5|            0|          0|   N|
|R1XIBC6WQBW31M|          5|            

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/<database name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

NameError: ignored

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)